In [1]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [2]:
with open('data.json', 'r') as file:
    documents = json.load(file)

sentences = [document['text'] for document in documents ]
sentences

['A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'H

In [3]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
embeddings = model.encode(sentences)

vector_size=embeddings.shape[1]
vector_size


768

In [4]:
def encode(text):
    return model.encode(text)

encode('test tset test')

array([ 0.09267613, -0.2954713 ,  0.5402585 ,  0.4545256 ,  0.02989052,
       -0.06390252,  1.0432135 ,  0.285783  ,  0.48432446, -0.89882034,
        0.04629606, -0.4880018 ,  0.6110357 , -0.67543715, -0.02487973,
       -0.41491058,  0.3928949 , -0.3814732 ,  0.15385595, -0.45826188,
        0.16211057, -0.509325  , -0.1309506 , -0.07078505, -0.52277917,
       -0.33120352,  0.796622  ,  0.87779284, -0.3819348 , -0.00200451,
        0.19375099, -0.306562  , -0.7208682 ,  1.070737  ,  0.10654572,
        0.05970209, -0.08060791,  0.60261893, -0.74010485, -0.22304766,
        0.24573548, -0.66291744, -0.05033381, -0.6192549 , -1.2056552 ,
        0.08224203, -1.133695  , -0.65495026, -0.96483344,  0.56101733,
        0.5568648 ,  0.6540963 , -0.2230277 , -0.8906296 , -0.07321373,
        0.22608165, -0.72347856,  0.13155252, -0.46265802,  0.57513505,
       -0.24697585,  0.00237589, -0.40968123, -0.09934282,  0.5272143 ,
       -0.18895681,  0.46983242,  0.11003574,  0.47173014, -0.55

In [5]:
# Create a flat Faiss index
index = faiss.IndexIDMap(faiss.IndexFlatIP(vector_size)) # the size of our vector space

# Add the document vectors into the index. They need to be transformed into numpy arrays first
index.add_with_ids(
    embeddings,
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(sentences)))
)

In [6]:
# save index to file
faiss.write_index(index, "faiss.index")

In [6]:
def search(query: str, k=5):
    encoded_query = model.encode(query).reshape((1, -1))
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [documents[_id]['title'] for _id in top_k[1][0]]
    return list(zip(results, scores))

In [7]:
with open('questions.json') as f:
    questions = json.load(f)
    
for question in questions:
    print("Question: ", question)
    answers = search(question, 5)
    for i, answer in enumerate(answers):
        print(f"Answer{i}: ",  answer)

Question:  How many people have died during Black Death?
Answer0:  ('Pandemic', 44.64852)
Answer1:  ('Spanish flu', 42.81655)
Answer2:  ('Pandemic prevention', 40.31984)
Answer3:  ('HIV/AIDS', 39.416985)
Answer4:  ('1929–1930 psittacosis pandemic', 36.452164)
Question:  Which diseases can be transmitted by animals?
Answer0:  ('Swine influenza', 89.769066)
Answer1:  ('Targeted immunization strategies', 78.843094)
Answer2:  ('Pandemic prevention', 71.772575)
Answer3:  ('HIV/AIDS', 67.65446)
Answer4:  ('PREDICT (USAID)', 61.000828)
Question:  Connection between climate change and a likelihood of a pandemic
Answer0:  ('Pandemic prevention', 117.75093)
Answer1:  ('PREDICT (USAID)', 108.27203)
Answer2:  ('Pandemic severity index', 90.160446)
Answer3:  ('Pandemic', 88.39629)
Answer4:  ('Targeted immunization strategies', 85.431206)
Question:  What is an example of a latent virus
Answer0:  ('PREDICT (USAID)', 93.40964)
Answer1:  ('HIV/AIDS', 86.7935)
Answer2:  ('Pandemic prevention', 72.728424